In [ ]:
# Import

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle
import numpy.matlib
import scipy.io
import scipy.stats as stats
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib Figure settings
mpl.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 1
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

In [ ]:
# Load dataframe containing all units info and data

print('Loading data...')
with open('Data_4_Ephys.pickle', 'rb') as f:
    clustInfoAll = pickle.load(f)
print('Loading complete. Ready to continue.')

In [ ]:
# ANALYSIS

print('Processing...')

#Set hardcoded parameters

# Set sounds and AM stim to analyze
AMlist = np.array([0.0, 1.0])  # AM modulation depth 0 (unmodulated noise) or 1 (10 Hz AM 100% depth)
sounds = np.unique(np.array(clustInfoAll.at[0, 'dB_stim']))  # Sound levels in dB SPL

# Some additional parameters for plotting
edges = np.arange(-1.0,1.5,.01)  # starting time, ending time, binSize - for raster and psth
time = edges[:-1] + np.diff(edges).mean()/2  # time vector 
# Create function 'smooth' for smoothing PSTHs
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

# Define time windows to use
SoundDelay = 0.250  # Sound onset delay after odor valve opening
BB_SoundDuration = 0.250  # Sound duration window to analyze for unmodulated broadband noise stim
BB_BLtime = (time>(-1)) & (time<0)  # Baseline window to analyze for unmodulated broadband noise stim z-scoring
AM_SoundDuration = 1.0  # Sound duration window to analyze for 10 Hz AM noise stim
AM_BLtime = (time>(-1)) & (time<0)  # Baseline window to analyze for 10 Hz AM noise stim z-scoring


# Run GLMs

# BB stim (unmodulated broadband noise)
AM = 0.0
soundtime = (time>SoundDelay) & (time<(SoundDelay+BB_SoundDuration))  # time window to analyze responses
# create new dataframe columns to fill in GLM results
clustInfoAll['GLM_BB_LEDoff_const_coef'] = np.nan
clustInfoAll['GLM_BB_LEDoff_dB_coef'] = np.nan
clustInfoAll['GLM_BB_LEDoff_Odor_coef'] = np.nan
clustInfoAll['GLM_BB_LEDoff_Interaction_coef'] = np.nan
clustInfoAll['GLM_BB_LEDoff_const_p'] = np.nan
clustInfoAll['GLM_BB_LEDoff_dB_p'] = np.nan
clustInfoAll['GLM_BB_LEDoff_Odor_p'] = np.nan
clustInfoAll['GLM_BB_LEDoff_Interaction_p'] = np.nan
clustInfoAll['GLM_BB_OMYN'] = np.nan
# Get individual unit stimuli and responses and run GLM 
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    indspikecountPSTH = indPSTH*0.01
    indspikesum = indspikecountPSTH[:, soundtime].sum(axis=1)
    AMmask = (indAM == AM)
    indspikesum = indspikesum[AMmask]
    inddB = inddB[AMmask]
    indVial = indVial[AMmask]
    if indspikesum.sum() == 0:
        continue
    inddB_new = np.zeros(len(inddB))
    indVial_new = np.zeros(len(indVial))
    inddB_new = inddB / np.max(inddB)
    for j,v in enumerate(np.unique(indVial)):
        indVial_new[indVial==v] = j+1
    indVial_new = indVial_new-1
    X = np.asarray([inddB_new, indVial_new, inddB_new*indVial_new])
    X = np.transpose(X)
    Xdf = pd.DataFrame(X, columns= ['dB', 'Odor', 'Interaction'])
    Xconst = sm.add_constant(Xdf)
    GLM_model = sm.GLM(indspikesum, Xconst, family=sm.families.Poisson())
    GLM_results = GLM_model.fit()
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_const_coef'] = GLM_results.params['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_const_p'] = GLM_results.pvalues['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_dB_coef'] = GLM_results.params['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_dB_p'] = GLM_results.pvalues['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_Odor_coef'] = GLM_results.params['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_Odor_p'] = GLM_results.pvalues['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_Interaction_coef'] = GLM_results.params['Interaction']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_Interaction_p'] = GLM_results.pvalues['Interaction']
    if (GLM_results.pvalues['dB']<0.05) & (GLM_results.params['dB']>0) & ((GLM_results.pvalues['Odor']<0.05) | (GLM_results.pvalues['Interaction']<0.05)):
        clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_OMYN'] = 1
    else:
        clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_OMYN'] = 0

# AM stim (10 Hz AM noise)
AM = 1.0
soundtime = (time>SoundDelay) & (time<(SoundDelay+AM_SoundDuration))  # time window to analyze responses
# create new dataframe columns to fill in GLM results
clustInfoAll['GLM_AM_LEDoff_const_coef'] = np.nan
clustInfoAll['GLM_AM_LEDoff_dB_coef'] = np.nan
clustInfoAll['GLM_AM_LEDoff_Odor_coef'] = np.nan
clustInfoAll['GLM_AM_LEDoff_Interaction_coef'] = np.nan
clustInfoAll['GLM_AM_LEDoff_const_p'] = np.nan
clustInfoAll['GLM_AM_LEDoff_dB_p'] = np.nan
clustInfoAll['GLM_AM_LEDoff_Odor_p'] = np.nan
clustInfoAll['GLM_AM_LEDoff_Interaction_p'] = np.nan
clustInfoAll['GLM_AM_OMYN'] = np.nan
# Get individual unit stimuli and responses and run GLM 
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    indspikecountPSTH = indPSTH*0.01
    indspikesum = indspikecountPSTH[:, soundtime].sum(axis=1)
    AMmask = (indAM == AM)
    indspikesum = indspikesum[AMmask]
    inddB = inddB[AMmask]
    indVial = indVial[AMmask]
    if indspikesum.sum() == 0:
        continue
    inddB_new = np.zeros(len(inddB))
    indVial_new = np.zeros(len(indVial))
    inddB_new = inddB / np.max(inddB)
    for j,v in enumerate(np.unique(indVial)):
        indVial_new[indVial==v] = j+1
    indVial_new = indVial_new-1
    X = np.asarray([inddB_new, indVial_new, inddB_new*indVial_new])
    X = np.transpose(X)
    Xdf = pd.DataFrame(X, columns= ['dB', 'Odor', 'Interaction'])
    Xconst = sm.add_constant(Xdf)
    GLM_model = sm.GLM(indspikesum, Xconst, family=sm.families.Poisson())
    GLM_results = GLM_model.fit()
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_const_coef'] = GLM_results.params['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_const_p'] = GLM_results.pvalues['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_dB_coef'] = GLM_results.params['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_dB_p'] = GLM_results.pvalues['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_Odor_coef'] = GLM_results.params['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_Odor_p'] = GLM_results.pvalues['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_Interaction_coef'] = GLM_results.params['Interaction']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_Interaction_p'] = GLM_results.pvalues['Interaction']
    if (GLM_results.pvalues['dB']<0.05) & (GLM_results.params['dB']>0) & ((GLM_results.pvalues['Odor']<0.05) | (GLM_results.pvalues['Interaction']<0.05)):
        clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_OMYN'] = 1
    else:
        clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_OMYN'] = 0


# ANALYSIS

# BB stim (unmodulated broadband noise)
# PSTHs, Responses, OMIs
AM = 0.0
BLtime =  BB_BLtime  # Baseline time window
soundtime = (time>SoundDelay) & (time<(SoundDelay+BB_SoundDuration))  # Response time window
# create dataframe columns
for dB in sounds:
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'].astype(object)
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'].astype(object)           
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['OMI_'+str(dB)+'_'+str(AM)+'_NoLED'] = np.nan
    clustInfoAll['Delta_'+str(dB)+'_'+str(AM)+'_NoLED'] = np.nan
# individual unit analysis
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    for dB in sounds:
        NoOdorNoLEDmask = (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorNoLEDtrials = indPSTH[NoOdorNoLEDmask,:]
        NoOdorNoLEDresps = NoOdorNoLEDtrials[:,soundtime].mean(axis=1)
        NoOdorpsthNoLED = indPSTH[NoOdorNoLEDmask,:].mean(axis=0)
        OdorNoLEDmask = (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorNoLEDtrials = indPSTH[OdorNoLEDmask,:]
        OdorNoLEDresps = OdorNoLEDtrials[:,soundtime].mean(axis=1)
        OdorpsthNoLED = indPSTH[OdorNoLEDmask,:].mean(axis=0)
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = [NoOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = [OdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = NoOdorpsthNoLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = OdorpsthNoLED[soundtime].mean()
        # OMI = Odor Modulation Index
        OMI_LEDoff = (OdorpsthNoLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean()) / (OdorpsthNoLED[soundtime].mean() + NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'OMI_'+str(dB)+'_'+str(AM)+'_NoLED'] = OMI_LEDoff
        # Delta = Odor - NoOdor
        Delta_LEDoff = (OdorpsthNoLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'Delta_'+str(dB)+'_'+str(AM)+'_NoLED'] = Delta_LEDoff
# make arrays for each unit for avg sound responses (FR_dB_Plot), for Odor and NoOdor, and add to dataframe
# create dataframe columns
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'].astype(object)
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'].astype(object)
# get individual PSTHs
for ind in clustInfoAll.index:
    dBarrayNoLED_NoOdor = []
    dBarrayNoLED_Odor = []       
    for dB in sounds:
        psthNoLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
        meanFR_NoLED_NoOdor = psthNoLED_NoOdor[soundtime].mean()
        dBarrayNoLED_NoOdor = np.append(dBarrayNoLED_NoOdor, meanFR_NoLED_NoOdor)
        psthNoLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
        meanFR_NoLED_Odor = psthNoLED_Odor[soundtime].mean()
        dBarrayNoLED_Odor = np.append(dBarrayNoLED_Odor, meanFR_NoLED_Odor)
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = [dBarrayNoLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = [dBarrayNoLED_Odor]
# Z scoring
# Get BLmean and BLstd for indiv neuron z-scoring
clustInfoAll['BL_FRmean'] = np.nan
clustInfoAll['BL_FRstd'] = np.nan
# create dataframe columns for Z-scored PSTHs
for dB in sounds:
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'].astype(object)
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'].astype(object)           
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    BLalltrials = indPSTH[:,BLtime].mean(axis=0)
    BLmean = BLalltrials.mean()
    BLstd = BLalltrials.std()
    clustInfoAll.at[clustInfoAll.index[ind], 'BL_FRmean'] = BLmean
    clustInfoAll.at[clustInfoAll.index[ind], 'BL_FRstd'] = BLstd
    for dB in sounds:
        NoOdorNoLEDmask = (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorpsthNoLED = indPSTH[NoOdorNoLEDmask,:].mean(axis=0)
        OdorNoLEDmask = (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorpsthNoLED = indPSTH[OdorNoLEDmask,:].mean(axis=0)
        zNoOdorpsthNoLED = (NoOdorpsthNoLED - (BLmean)) / BLstd
        zOdorpsthNoLED = (OdorpsthNoLED - (BLmean)) / BLstd
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = [zNoOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = [zOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = zNoOdorpsthNoLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = zOdorpsthNoLED[soundtime].mean()
# make arrays for each unit for Z-Scored avg sound responses (zFR_dB_Plot), for Odor and NoOdor, and add to dataframe
# create dataframe columns
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'].astype(object)
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'].astype(object)
# get individual PSTHs
for ind in clustInfoAll.index:
    dBarrayNoLED_NoOdor = []
    dBarrayNoLED_Odor = []       
    for dB in sounds:
        psthNoLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
        meanFR_NoLED_NoOdor = psthNoLED_NoOdor[soundtime].mean()
        dBarrayNoLED_NoOdor = np.append(dBarrayNoLED_NoOdor, meanFR_NoLED_NoOdor)
        psthNoLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
        meanFR_NoLED_Odor = psthNoLED_Odor[soundtime].mean()
        dBarrayNoLED_Odor = np.append(dBarrayNoLED_Odor, meanFR_NoLED_Odor)
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = [dBarrayNoLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = [dBarrayNoLED_Odor]
# Get units that are overall positively or negatively modulated by odor (Overall OMI: responses to 55, 65, 75 dB)
clustInfoAll['OdorEffect_v_'+str(AM)+'_NoLED'] = np.nan  # Overall OMI
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    Soundmeans = indPSTH[:,soundtime].mean(axis=1)
    NoLED_NoOdor_mask = (indVial == NoOdorVial) & (inddB >50) & (indAM == AM)
    NoLED_Odor_mask = (indVial == OdorVial) & (inddB >50) & (indAM == AM)
    NoLED_NoOdor_Resp = Soundmeans[NoLED_NoOdor_mask]
    NoLED_Odor_Resp = Soundmeans[NoLED_Odor_mask]
    OdorEffectvalue = (NoLED_Odor_Resp.mean() - NoLED_NoOdor_Resp.mean()) / (NoLED_Odor_Resp.mean() + NoLED_NoOdor_Resp.mean())
    clustInfoAll.at[clustInfoAll.index[ind], 'OdorEffect_v_'+str(AM)+'_NoLED'] = OdorEffectvalue

# AM stim (10 Hz AM noise)
# PSTHs, Responses, OMIs
AM = 1.0
BLtime =  AM_BLtime  # Baseline time window
soundtime = (time>SoundDelay) & (time<(SoundDelay+AM_SoundDuration))  # Response time window
# create dataframe columns
for dB in sounds:
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'].astype(object)
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'].astype(object)           
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['OMI_'+str(dB)+'_'+str(AM)+'_NoLED'] = np.nan
    clustInfoAll['Delta_'+str(dB)+'_'+str(AM)+'_NoLED'] = np.nan
# individual unit analysis
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    for dB in sounds:
        NoOdorNoLEDmask = (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorNoLEDtrials = indPSTH[NoOdorNoLEDmask,:]
        NoOdorNoLEDresps = NoOdorNoLEDtrials[:,soundtime].mean(axis=1)
        NoOdorpsthNoLED = indPSTH[NoOdorNoLEDmask,:].mean(axis=0)
        OdorNoLEDmask = (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorNoLEDtrials = indPSTH[OdorNoLEDmask,:]
        OdorNoLEDresps = OdorNoLEDtrials[:,soundtime].mean(axis=1)
        OdorpsthNoLED = indPSTH[OdorNoLEDmask,:].mean(axis=0)
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = [NoOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = [OdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = NoOdorpsthNoLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = OdorpsthNoLED[soundtime].mean()
        # OMI = Odor Modulation Index
        OMI_LEDoff = (OdorpsthNoLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean()) / (OdorpsthNoLED[soundtime].mean() + NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'OMI_'+str(dB)+'_'+str(AM)+'_NoLED'] = OMI_LEDoff
        # Delta = Odor - NoOdor
        Delta_LEDoff = (OdorpsthNoLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'Delta_'+str(dB)+'_'+str(AM)+'_NoLED'] = Delta_LEDoff
# make arrays for each unit for avg sound responses (FR_dB_Plot), for Odor and NoOdor, and add to dataframe
# create dataframe columns
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'].astype(object)
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'].astype(object)
# get individual PSTHs
for ind in clustInfoAll.index:
    dBarrayNoLED_NoOdor = []
    dBarrayNoLED_Odor = []       
    for dB in sounds:
        psthNoLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
        meanFR_NoLED_NoOdor = psthNoLED_NoOdor[soundtime].mean()
        dBarrayNoLED_NoOdor = np.append(dBarrayNoLED_NoOdor, meanFR_NoLED_NoOdor)
        psthNoLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
        meanFR_NoLED_Odor = psthNoLED_Odor[soundtime].mean()
        dBarrayNoLED_Odor = np.append(dBarrayNoLED_Odor, meanFR_NoLED_Odor)
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = [dBarrayNoLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = [dBarrayNoLED_Odor]
# Z scoring
# Get BLmean and BLstd for indiv neuron z-scoring
clustInfoAll['BL_FRmean'] = np.nan
clustInfoAll['BL_FRstd'] = np.nan
# create dataframe columns for Z-scored PSTHs
for dB in sounds:
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'].astype(object)
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'].astype(object)           
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    BLalltrials = indPSTH[:,BLtime].mean(axis=0)
    BLmean = BLalltrials.mean()
    BLstd = BLalltrials.std()
    clustInfoAll.at[clustInfoAll.index[ind], 'BL_FRmean'] = BLmean
    clustInfoAll.at[clustInfoAll.index[ind], 'BL_FRstd'] = BLstd
    for dB in sounds:
        NoOdorNoLEDmask = (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorpsthNoLED = indPSTH[NoOdorNoLEDmask,:].mean(axis=0)
        OdorNoLEDmask = (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorpsthNoLED = indPSTH[OdorNoLEDmask,:].mean(axis=0)
        zNoOdorpsthNoLED = (NoOdorpsthNoLED - (BLmean)) / BLstd
        zOdorpsthNoLED = (OdorpsthNoLED - (BLmean)) / BLstd
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = [zNoOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = [zOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = zNoOdorpsthNoLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = zOdorpsthNoLED[soundtime].mean()
# make arrays for each unit for Z-Scored avg sound responses (zFR_dB_Plot), for Odor and NoOdor, and add to dataframe
# create dataframe columns
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'].astype(object)
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'].astype(object)
# get individual PSTHs
for ind in clustInfoAll.index:
    dBarrayNoLED_NoOdor = []
    dBarrayNoLED_Odor = []       
    for dB in sounds:
        psthNoLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
        meanFR_NoLED_NoOdor = psthNoLED_NoOdor[soundtime].mean()
        dBarrayNoLED_NoOdor = np.append(dBarrayNoLED_NoOdor, meanFR_NoLED_NoOdor)
        psthNoLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
        meanFR_NoLED_Odor = psthNoLED_Odor[soundtime].mean()
        dBarrayNoLED_Odor = np.append(dBarrayNoLED_Odor, meanFR_NoLED_Odor)
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = [dBarrayNoLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = [dBarrayNoLED_Odor]
# Get units that are overall positively or negatively modulated by odor (Overall OMI: responses to 55, 65, 75 dB)
clustInfoAll['OdorEffect_v_'+str(AM)+'_NoLED'] = np.nan  # Overall OMI
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    Soundmeans = indPSTH[:,soundtime].mean(axis=1)
    NoLED_NoOdor_mask = (indVial == NoOdorVial) & (inddB >50) & (indAM == AM)
    NoLED_Odor_mask = (indVial == OdorVial) & (inddB >50) & (indAM == AM)
    NoLED_NoOdor_Resp = Soundmeans[NoLED_NoOdor_mask]
    NoLED_Odor_Resp = Soundmeans[NoLED_Odor_mask]
    OdorEffectvalue = (NoLED_Odor_Resp.mean() - NoLED_NoOdor_Resp.mean()) / (NoLED_Odor_Resp.mean() + NoLED_NoOdor_Resp.mean())
    clustInfoAll.at[clustInfoAll.index[ind], 'OdorEffect_v_'+str(AM)+'_NoLED'] = OdorEffectvalue


# Create function 'bar_plot' for graphing bar plots with multiple groups
# adapted from https://stackoverflow.com/questions/14270391/how-to-plot-multiple-bars-grouped
def bar_plot(ax, data, errdata, colors=None, total_width=0.8, single_width=1, legend=True):
    """Draws a bar plot with multiple bars per data point.
    Parameters
    ----------
    ax : matplotlib.pyplot.axis
        The axis we want to draw our plot on.
    data: dictionary
        A dictionary containing the data we want to plot. Keys are the names of the
        data, the items is a list of the values.
        Example:
        data = {
            "x":[1,2,3],
            "y":[1,2,3],
            "z":[1,2,3],
        }
    errdata: dictionary containing the errorbar data, with keys matching the corresponding data table
    colors : array-like, optional
        A list of colors which are used for the bars. If None, the colors
        will be the standard matplotlib color cyle. (default: None)
    total_width : float, optional, default: 0.8
        The width of a bar group. 0.8 means that 80% of the x-axis is covered
        by bars and 20% will be spaces between the bars.
    single_width: float, optional, default: 1
        The relative width of a single bar within a group. 1 means the bars
        will touch eachother within a group, values less than 1 will make
        these bars thinner.
    legend: bool, optional, default: True
        If this is set to true, a legend will be added to the axis.
    """
    # Check if colors where provided, otherwhise use the default color cycle
    if colors is None:
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    # Number of bars per group
    n_bars = len(data)
    # The width of a single bar
    bar_width = total_width / n_bars
    # List containing handles for the drawn bars, used for the legend
    bars = []
    # Iterate over all data
    for i, (name, values) in enumerate(data.items()):
        # The offset in x direction of that bar
        x_offset = (i - n_bars / 2) * bar_width + bar_width / 2
        # Draw a bar for every value of that type
        for x, y in enumerate(values):
            bar = ax.bar(x + x_offset, y, width=bar_width * single_width, color=colors[i % len(colors)], 
                         edgecolor=['black', 'black'], yerr=errdata[str(name)][x], capsize=10, fill=True)
        # Add a handle to the last drawn bar, which we'll need for the legend
        bars.append(bar[0])
    # Draw legend if we need
    if legend:
        ax.legend(bars, data.keys())


print('Analysis complete. Ready to continue.')

In [ ]:
# FIGURE 4 A-D

fig, axs = plt.subplots(nrows=4,ncols=2,figsize=(12, 24), sharex=False)
fig.subplots_adjust(hspace=.4,wspace=0.3)
# BB stim (unmodulated broadband noise)
AM = 0.0
clustInfoMask = clustInfoAll.loc[((clustInfoAll['GLM_BB_LEDoff_dB_p']<0.05) &
                             (clustInfoAll['GLM_BB_LEDoff_dB_coef']>0) &      
                            (clustInfoAll['BL_FRmean']>1) &
                                ((clustInfoAll['GLM_BB_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_BB_LEDoff_Interaction_p']<0.05)))]
total_n = len(clustInfoMask.index)
mouse_N = len(clustInfoMask.mouse.unique())
print(str(mouse_N)+ ' mice')
print(clustInfoMask.mouse.unique())
print(str(total_n)+ ' total units')
clustInfoENH = clustInfoMask.loc[((clustInfoMask['OdorEffect_v_'+str(AM)+'_NoLED']>=0))]
clustInfoSUPP = clustInfoMask.loc[((clustInfoMask['OdorEffect_v_'+str(AM)+'_NoLED']<0))]
Enh_n = len(clustInfoENH.index)
print(str(Enh_n)+' Enh. units')
Supp_n = len(clustInfoSUPP.index)
print(str(Supp_n)+' Supp. units')
#################################################################################################################
#### PLOT 1 : GLM PIE CHART ODOR MOD - 4A LEFT
plt.subplot(4,2,1)
N_total = clustInfoAll.shape[0]
Sig_sound = clustInfoAll.loc[((clustInfoAll['GLM_BB_LEDoff_dB_p']<0.05) &
                              (clustInfoAll['BL_FRmean']>1) &
                                (clustInfoAll['GLM_BB_LEDoff_dB_coef']>0))]
N_Sig_sound = Sig_sound.shape[0]
print(Sig_sound.mouse.unique())
print(str(N_Sig_sound)+ ' sound-responsive units')
Sig_odor = clustInfoAll.loc[((clustInfoAll['GLM_BB_LEDoff_dB_p']<0.05) &
                            (clustInfoAll['GLM_BB_LEDoff_dB_coef']>0) &
                             (clustInfoAll['BL_FRmean']>1) &
                                ((clustInfoAll['GLM_BB_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_BB_LEDoff_Interaction_p']<0.05)))]
N_Sig_odor = Sig_odor.shape[0]
print(str(N_Sig_odor)+ ' odor mod. units')
Pct_soundonly = N_Sig_sound - N_Sig_odor
print(str(Pct_soundonly)+ ' sound only units')
Pct_Odor = N_Sig_odor
print(Pct_Odor)
wedges = [Pct_soundonly, Pct_Odor]
plt.pie(wedges, explode = [0, 0.1], labels = ['Sound only', 'Odor mod.'], autopct='%1.1f%%', colors=['gray', 'magenta'], startangle=90)
plt.title(str(N_Sig_sound)+' sound-responsive units')
#################################################################################################################
#### PLOT 2 : PIE CHART ENH SUPP - 4A RIGHT
plt.subplot(4,2,2)
N_total = clustInfoAll.shape[0]
Sig_Enh = clustInfoAll.loc[((clustInfoAll['GLM_BB_LEDoff_dB_p']<0.05) &
                            (clustInfoAll['GLM_BB_LEDoff_dB_coef']>0) &
                            (clustInfoAll['BL_FRmean']>1) &
                            (clustInfoAll['OdorEffect_v_'+str(AM)+'_NoLED']>=0) &
                                ((clustInfoAll['GLM_BB_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_BB_LEDoff_Interaction_p']<0.05)))]
N_Sig_Enh = Sig_Enh.shape[0]
print(Sig_Enh.mouse.unique())
print(str(N_Sig_Enh)+ ' enhanced responses')
Sig_Supp = clustInfoAll.loc[((clustInfoAll['GLM_BB_LEDoff_dB_p']<0.05) &
                            (clustInfoAll['GLM_BB_LEDoff_dB_coef']>0) &
                             (clustInfoAll['BL_FRmean']>1) &
                            (clustInfoAll['OdorEffect_v_'+str(AM)+'_NoLED']<0) &
                                ((clustInfoAll['GLM_BB_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_BB_LEDoff_Interaction_p']<0.05)))]
N_Sig_Supp = Sig_Supp.shape[0]
print(str(N_Sig_Supp)+ ' suppressed responses')
wedges = [N_Sig_Enh, N_Sig_Supp]
plt.pie(wedges, explode = [0, 0.1], labels = ['Enhanced', 'Suppressed'], autopct='%1.1f%%', colors=['mediumvioletred', 'plum'], startangle=90)
plt.title(str(N_Sig_Enh+N_Sig_Supp)+' odor modulated units')
#################################################################################################################
#### PLOT 3 : ENHANCED DB LINE PLOT - 4B LEFT
plt.subplot(4,2,3)
mean_dBarray_NoLED_NoOdor = []
mean_dBarray_NoLED_Odor = []
sem_dBarray_NoLED_NoOdor = []
sem_dBarray_NoLED_Odor = []
soundticks = [0, 1, 2, 3]
print('ENHANCED STATS LINE PLOT:')
for i, dB in enumerate(np.asarray(sounds)):
    NoLED_NoOdor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
    NoLED_Odor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
    t, p = stats.shapiro(NoLED_NoOdor)
    print(str(dB)+': NoOdor shap. p= '+str(p))
    t, pp = stats.shapiro(NoLED_Odor)
    print(str(dB)+': Odor shap. p= '+str(pp))
    if p<0.05 or pp<0.05:
        t, p = stats.wilcoxon(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' Wilcoxon Z= '+str(t))
        print(str(dB)+' Wilcoxon p= '+str(p))
    else:
        t, p = stats.ttest_re;l(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' Paired ttest t= '+str(t))
        print(str(dB)+' Paired ttest p= '+str(p))
    mean_dBarray_NoLED_NoOdor = np.append(mean_dBarray_NoLED_NoOdor, NoLED_NoOdor.mean())
    mean_dBarray_NoLED_Odor = np.append(mean_dBarray_NoLED_Odor, NoLED_Odor.mean())
    sem_dBarray_NoLED_NoOdor = np.append(sem_dBarray_NoLED_NoOdor, NoLED_NoOdor.std()/np.sqrt(len(NoLED_NoOdor)))
    sem_dBarray_NoLED_Odor = np.append(sem_dBarray_NoLED_Odor, NoLED_Odor.std()/np.sqrt(len(NoLED_Odor)))
plt.errorbar(sounds[1:], mean_dBarray_NoLED_NoOdor[1:], sem_dBarray_NoLED_NoOdor[1:], capsize=5, marker='o', color='black')
plt.errorbar(sounds[1:], mean_dBarray_NoLED_Odor[1:], sem_dBarray_NoLED_Odor[1:], capsize=5, marker='o', color='mediumvioletred')
plt.legend(['Sound (control)', 'Odor+Sound'], loc='upper left')
plt.ylim([0, 25])
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(sounds[1:], labels=sounds[1:]-50)
plt.ylabel('Avg. Firing Rate (Hz) Z-scored')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.title('Enhanced')
#################################################################################################################
#### PLOT 4 : SUPPRESSED DB LINE PLOT - 4B RIGHT
plt.subplot(4,2,4)
mean_dBarray_NoLED_NoOdor = []
mean_dBarray_NoLED_Odor = []
sem_dBarray_NoLED_NoOdor = []
sem_dBarray_NoLED_Odor = []
print('SUPPRESSED STATS LINE PLOT:')
for i, dB in enumerate(np.asarray(sounds)):
    NoLED_NoOdor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
    NoLED_Odor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
    t, p = stats.shapiro(NoLED_NoOdor)
    print(str(dB)+': NoOdor shap. p= '+str(p))
    t, pp = stats.shapiro(NoLED_Odor)
    print(str(dB)+': Odor shap. p= '+str(pp))
    if p<0.05 or pp<0.05:
        t, p = stats.wilcoxon(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' Wilcoxon Z= '+str(t))
        print(str(dB)+' Wilcoxon p= '+str(p))
    else:
        t, p = stats.ttest_re;l(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' Paired ttest t= '+str(t))
        print(str(dB)+' Paired ttest p= '+str(p))
    mean_dBarray_NoLED_NoOdor = np.append(mean_dBarray_NoLED_NoOdor, NoLED_NoOdor.mean())
    mean_dBarray_NoLED_Odor = np.append(mean_dBarray_NoLED_Odor, NoLED_Odor.mean())
    sem_dBarray_NoLED_NoOdor = np.append(sem_dBarray_NoLED_NoOdor, NoLED_NoOdor.std()/np.sqrt(len(NoLED_NoOdor)))
    sem_dBarray_NoLED_Odor = np.append(sem_dBarray_NoLED_Odor, NoLED_Odor.std()/np.sqrt(len(NoLED_Odor)))
plt.errorbar(sounds[1:], mean_dBarray_NoLED_NoOdor[1:], sem_dBarray_NoLED_NoOdor[1:], capsize=5, marker='o', color='black')
plt.errorbar(sounds[1:], mean_dBarray_NoLED_Odor[1:], sem_dBarray_NoLED_Odor[1:], capsize=5, marker='o', color='plum')
plt.legend(['Sound (control)', 'Odor+Sound'], loc='upper left')
plt.ylim([0, 25])
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(sounds[1:], labels=sounds[1:]-50)
plt.ylabel('Avg. Firing Rate (Hz) Z-scored')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.title('Suppressed')
#################################################################################################################
#### PLOT 5 : ENHANCED STATS COMPARISON BAR - 4C LEFT
plt.subplot(4,2,5)
mean_OMIarray_NoLED = []
sem_OMIarray_NoLED = []
K_data = {}
print('ENHANCED STATS FRIEDMAN:')
for i, dB in enumerate(np.asarray(sounds[1:])):
    NoLED = np.array((clustInfoENH['OMI_'+str(dB)+'_'+str(AM)+'_NoLED']))
    K_data[str(dB)] = NoLED[~np.isnan(NoLED)]
    mean_OMIarray_NoLED = np.append(mean_OMIarray_NoLED, np.nanmean(NoLED))
    sem_OMIarray_NoLED = np.append(sem_OMIarray_NoLED, np.nanstd(NoLED)/np.sqrt(len(NoLED))) 
h, p = stats.friedmanchisquare(K_data['45'], K_data['55'], K_data['65'], K_data['75'])
print('Enhanced Friedman test Q= '+str(h))
print('Enhanced Friedman test p= '+str(p))
data = {
        "LED off": mean_OMIarray_NoLED
    }
errdata = {
        "LED off": sem_OMIarray_NoLED
    }
bar_plot(axs[2,0], data, errdata, colors=['mediumvioletred'], total_width=.8, single_width=.9, legend=False)
plt.ylabel('OMI')
plt.xlabel('Sound Level (dB SNR)')
plt.xticks([0, 1, 2, 3], labels=sounds[1:]-50)
plt.ylim([-0.2, 0.2])     #OMI
plt.yticks([-0.2, 0, 0.2])     #OMI
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 6 : SUPPRESSED STATS COMPARISON BAR - 4C RIGHT
plt.subplot(4,2,6)
mean_OMIarray_NoLED = []
sem_OMIarray_NoLED = []
K_data = {}
print('SUPPRESSED STATS FRIEDMAN:')
for i, dB in enumerate(np.asarray(sounds[1:])):
    NoLED = np.array((clustInfoSUPP['OMI_'+str(dB)+'_'+str(AM)+'_NoLED']))
    K_data[str(dB)] = NoLED
    mean_OMIarray_NoLED = np.append(mean_OMIarray_NoLED, NoLED.mean())
    sem_OMIarray_NoLED = np.append(sem_OMIarray_NoLED, NoLED.std()/np.sqrt(len(NoLED)))
h, p = stats.friedmanchisquare(K_data['45'], K_data['55'], K_data['65'], K_data['75'])
print('Suppressed Friedman test Q= '+str(h))
print('Suppressed Friedman test p= '+str(p))
data = {
        "LED off": mean_OMIarray_NoLED
    }
errdata = {
        "LED off": sem_OMIarray_NoLED
    }
bar_plot(axs[2,1], data, errdata, colors=['plum'], total_width=.8, single_width=.9, legend=False)
plt.ylabel('OMI')
plt.xlabel('Sound Level (dB SNR)')
plt.xticks([0, 1, 2, 3], labels=sounds[1:]-50)
plt.ylim([-0.2, 0.2])     #OMI
plt.yticks([-0.2, 0, 0.2])     #OMI
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 7 : ENHANCED PSTH - 4D LEFT
plt.subplot(4,2,7)
dB_sel = 55
clustInfoENH = clustInfoMask.loc[(clustInfoMask['Delta_'+str(dB_sel)+'_'+str(AM)+'_NoLED']>0)]
n = len(clustInfoENH.index)
PSTHall_NoLED_NoOdor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_NoOdor'])))
PSTHall_NoLED_Odor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_Odor'])))
meanNoLED_NoOdor = PSTHall_NoLED_NoOdor.mean(axis=0)
semNoLED_NoOdor = PSTHall_NoLED_NoOdor.std(axis=0)/np.sqrt(n)
meanNoLED_Odor = PSTHall_NoLED_Odor.mean(axis=0)
semNoLED_Odor = PSTHall_NoLED_Odor.std(axis=0)/np.sqrt(n)
plt.plot(time, smooth(meanNoLED_NoOdor, 3), color='black')
plt.plot(time, smooth(meanNoLED_Odor, 3), color='mediumvioletred')
plt.legend(['Sound (control)','Odor+Sound'], loc='upper right')
plt.xlabel('Time (s)')
plt.ylabel('Avg. Firing Rate (Hz) Z-Scored')
plt.ylim(-5, 40)
plt.xlim(-0.25, 1.5)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.margins(x=0)
plt.xticks([0, 0.5, 1, 1.5])
plt.title(str(dB_sel-50)+ ' dB SNR')
plt.axvline(x=0, color='magenta', ls='--')
plt.axvline(x=0.250, color='gray', ls='--')
plt.axvline(x=1.250, color='gray', ls='--')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 8 : SUPPRESSED PSTH - 4D RIGHT
plt.subplot(4,2,8)
dB_sel = 65
clustInfoSUPP = clustInfoMask.loc[(clustInfoMask['Delta_'+str(dB_sel)+'_'+str(AM)+'_NoLED']<0)]
n = len(clustInfoSUPP.index)
PSTHall_NoLED_NoOdor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_NoOdor'])))
PSTHall_NoLED_Odor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_Odor'])))
meanNoLED_NoOdor = PSTHall_NoLED_NoOdor.mean(axis=0)
semNoLED_NoOdor = PSTHall_NoLED_NoOdor.std(axis=0)/np.sqrt(n)
meanNoLED_Odor = PSTHall_NoLED_Odor.mean(axis=0)
semNoLED_Odor = PSTHall_NoLED_Odor.std(axis=0)/np.sqrt(n)
plt.plot(time, smooth(meanNoLED_NoOdor, 3), color='black')
plt.plot(time, smooth(meanNoLED_Odor, 3), color='plum')
plt.legend(['Sound (control)','Odor+Sound'], loc='upper right')
plt.xlabel('Time (s)')
plt.ylabel('Avg. Firing Rate (Hz) Z-Scored')
plt.ylim(-5, 40)
plt.xlim(-0.25, 1.5)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.margins(x=0)
plt.xticks([0, 0.5, 1, 1.5])
plt.title(str(dB_sel-50)+ ' dB SNR')
plt.axvline(x=0, color='magenta', ls='--')
plt.axvline(x=0.250, color='gray', ls='--')
plt.axvline(x=1.250, color='gray', ls='--')
plt.axhline(y=0, color='black', ls='--')

# Save Fig
filename = 'Fig 4ABCD'+'.pdf'
plt.savefig(filename, bbox_inches='tight', dpi=300, transparent=False)

plt.show()

In [ ]:
# FIGURE 4 E-H

fig, axs = plt.subplots(nrows=4,ncols=2,figsize=(12, 24), sharex=False)
fig.subplots_adjust(hspace=.4,wspace=0.3)
# AM stim (10 Hz AM noise)
AM = 1.0
clustInfoMask = clustInfoAll.loc[((clustInfoAll['GLM_AM_LEDoff_dB_p']<0.05) &
                             (clustInfoAll['GLM_AM_LEDoff_dB_coef']>0) &    
                            (clustInfoAll['BL_FRmean']>1) &
                                ((clustInfoAll['GLM_AM_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_AM_LEDoff_Interaction_p']<0.05)))]
total_n = len(clustInfoMask.index)
mouse_N = len(clustInfoMask.mouse.unique())
print(str(mouse_N)+ ' mice')
print(clustInfoMask.mouse.unique())
print(str(total_n)+ ' total units')
clustInfoENH = clustInfoMask.loc[((clustInfoMask['OdorEffect_v_'+str(AM)+'_NoLED']>=0))]
clustInfoSUPP = clustInfoMask.loc[((clustInfoMask['OdorEffect_v_'+str(AM)+'_NoLED']<0))]
Enh_n = len(clustInfoENH.index)
print(str(Enh_n)+' Enh. units')
Supp_n = len(clustInfoSUPP.index)
print(str(Supp_n)+' Supp. units')
#################################################################################################################
#### PLOT 1 : GLM PIE CHART ODOR MOD - 4E LEFT
plt.subplot(4,2,1)
N_total = clustInfoAll.shape[0]
Sig_sound = clustInfoAll.loc[((clustInfoAll['GLM_AM_LEDoff_dB_p']<0.05) &
                              (clustInfoAll['BL_FRmean']>1) &
                                (clustInfoAll['GLM_AM_LEDoff_dB_coef']>0))]
N_Sig_sound = Sig_sound.shape[0]
print(Sig_sound.mouse.unique())
print(str(N_Sig_sound)+ ' sound-responsive units')
Sig_odor = clustInfoAll.loc[((clustInfoAll['GLM_AM_LEDoff_dB_p']<0.05) &
                            (clustInfoAll['GLM_AM_LEDoff_dB_coef']>0) &
                             (clustInfoAll['BL_FRmean']>1) &
                                ((clustInfoAll['GLM_AM_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_AM_LEDoff_Interaction_p']<0.05)))]
N_Sig_odor = Sig_odor.shape[0]
print(str(N_Sig_odor)+ ' odor mod. units')
Pct_soundonly = N_Sig_sound - N_Sig_odor
print(str(Pct_soundonly)+ ' sound only units')
Pct_Odor = N_Sig_odor
print(Pct_Odor)
wedges = [Pct_soundonly, Pct_Odor]
plt.pie(wedges, explode = [0, 0.1], labels = ['Sound only', 'Odor mod.'], autopct='%1.1f%%', colors=['gray', 'magenta'], startangle=90)
plt.title(str(N_Sig_sound)+' sound-responsive units')
#################################################################################################################
#### PLOT 2 : PIE CHART ENH SUPP - 4E RIGHT
plt.subplot(4,2,2)
N_total = clustInfoAll.shape[0]
Sig_Enh = clustInfoAll.loc[((clustInfoAll['GLM_AM_LEDoff_dB_p']<0.05) &
                            (clustInfoAll['GLM_AM_LEDoff_dB_coef']>0) &
                            (clustInfoAll['BL_FRmean']>1) &
                            (clustInfoAll['OdorEffect_v_'+str(AM)+'_NoLED']>=0) &
                                ((clustInfoAll['GLM_AM_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_AM_LEDoff_Interaction_p']<0.05)))]
N_Sig_Enh = Sig_Enh.shape[0]
print(Sig_Enh.mouse.unique())
print(str(N_Sig_Enh)+ ' enhanced responses')
Sig_Supp = clustInfoAll.loc[((clustInfoAll['GLM_AM_LEDoff_dB_p']<0.05) &
                            (clustInfoAll['GLM_AM_LEDoff_dB_coef']>0) &
                             (clustInfoAll['BL_FRmean']>1) &
                            (clustInfoAll['OdorEffect_v_'+str(AM)+'_NoLED']<0) &
                                ((clustInfoAll['GLM_AM_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_AM_LEDoff_Interaction_p']<0.05)))]
N_Sig_Supp = Sig_Supp.shape[0]
print(str(N_Sig_Supp)+ ' suppressed responses')
wedges = [N_Sig_Enh, N_Sig_Supp]
plt.pie(wedges, explode = [0, 0.1], labels = ['Enhanced', 'Suppressed'], autopct='%1.1f%%', colors=['mediumvioletred', 'plum'], startangle=90)
plt.title(str(N_Sig_Enh+N_Sig_Supp)+' odor modulated units')
#################################################################################################################
#### PLOT 3 : ENHANCED DB LINE PLOT - 4F LEFT
plt.subplot(4,2,3)
mean_dBarray_NoLED_NoOdor = []
mean_dBarray_NoLED_Odor = []
sem_dBarray_NoLED_NoOdor = []
sem_dBarray_NoLED_Odor = []
print('ENHANCED STATS LINE PLOT:')
for i, dB in enumerate(np.asarray(sounds)):
    NoLED_NoOdor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
    NoLED_Odor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
    t, p = stats.shapiro(NoLED_NoOdor)
    print(str(dB)+': NoOdor shap. p= '+str(p))
    t, pp = stats.shapiro(NoLED_Odor)
    print(str(dB)+': Odor shap. p= '+str(pp))
    if p<0.05 or pp<0.05:
        t, p = stats.wilcoxon(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' Wilcoxon Z= '+str(t))
        print(str(dB)+' Wilcoxon p= '+str(p))
    else:
        t, p = stats.ttest_rel(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' Paired ttest t= '+str(t))
        print(str(dB)+' Paired ttest p= '+str(p))
    mean_dBarray_NoLED_NoOdor = np.append(mean_dBarray_NoLED_NoOdor, NoLED_NoOdor.mean())
    mean_dBarray_NoLED_Odor = np.append(mean_dBarray_NoLED_Odor, NoLED_Odor.mean())
    sem_dBarray_NoLED_NoOdor = np.append(sem_dBarray_NoLED_NoOdor, NoLED_NoOdor.std()/np.sqrt(len(NoLED_NoOdor)))
    sem_dBarray_NoLED_Odor = np.append(sem_dBarray_NoLED_Odor, NoLED_Odor.std()/np.sqrt(len(NoLED_Odor)))
plt.errorbar(sounds[1:]-50, mean_dBarray_NoLED_NoOdor[1:], sem_dBarray_NoLED_NoOdor[1:], capsize=5, marker='o', color='black')
plt.errorbar(sounds[1:]-50, mean_dBarray_NoLED_Odor[1:], sem_dBarray_NoLED_Odor[1:], capsize=5, marker='o', color='mediumvioletred')
plt.legend(['Sound (control)', 'Odor+Sound'], loc='upper left')
plt.ylim([0, 20])
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(np.asarray(sounds[1:]-50))
plt.ylabel('Avg. Firing Rate (Hz) Z-scored')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.title('Enhanced')
#################################################################################################################
#### PLOT 4 : SUPPRESSED DB LINE PLOT - 4F RIGHT
plt.subplot(4,2,4)
mean_dBarray_NoLED_NoOdor = []
mean_dBarray_NoLED_Odor = []
sem_dBarray_NoLED_NoOdor = []
sem_dBarray_NoLED_Odor = []
print('SUPPRESSED STATS LINE PLOT:')
for i, dB in enumerate(np.asarray(sounds)):
    NoLED_NoOdor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
    NoLED_Odor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
    t, p = stats.shapiro(NoLED_NoOdor)
    print(str(dB)+': NoOdor shap. p= '+str(p))
    t, pp = stats.shapiro(NoLED_Odor)
    print(str(dB)+': Odor shap. p= '+str(pp))
    if p<0.05 or pp<0.05:
        t, p = stats.wilcoxon(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' Wilcoxon Z= '+str(t))
        print(str(dB)+' Wilcoxon p= '+str(p))
    else:
        t, p = stats.ttest_re;l(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' Paired ttest t= '+str(t))
        print(str(dB)+' Paired ttest p= '+str(p))
    mean_dBarray_NoLED_NoOdor = np.append(mean_dBarray_NoLED_NoOdor, NoLED_NoOdor.mean())
    mean_dBarray_NoLED_Odor = np.append(mean_dBarray_NoLED_Odor, NoLED_Odor.mean())
    sem_dBarray_NoLED_NoOdor = np.append(sem_dBarray_NoLED_NoOdor, NoLED_NoOdor.std()/np.sqrt(len(NoLED_NoOdor)))
    sem_dBarray_NoLED_Odor = np.append(sem_dBarray_NoLED_Odor, NoLED_Odor.std()/np.sqrt(len(NoLED_Odor)))
plt.errorbar(sounds[1:]-50, mean_dBarray_NoLED_NoOdor[1:], sem_dBarray_NoLED_NoOdor[1:], capsize=5, marker='o', color='black')
plt.errorbar(sounds[1:]-50, mean_dBarray_NoLED_Odor[1:], sem_dBarray_NoLED_Odor[1:], capsize=5, marker='o', color='plum')
plt.legend(['Sound (control)', 'Odor+Sound'], loc='upper left')
plt.ylim([0, 20])
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(np.asarray(sounds[1:]-50))
plt.ylabel('Avg. Firing Rate (Hz) Z-scored')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.title('Suppressed')
#################################################################################################################
#### PLOT 5 : ENHANCED STATS COMPARISON BAR - 4G LEFT
plt.subplot(4,2,5)
mean_OMIarray_NoLED = []
sem_OMIarray_NoLED = []
K_data = {}
print('ENHANCED STATS FRIEDMAN:')
for i, dB in enumerate(np.asarray(sounds[1:])):
    NoLED = np.array((clustInfoENH['OMI_'+str(dB)+'_'+str(AM)+'_NoLED']))
    K_data[str(dB)] = NoLED[~np.isnan(NoLED)]
    mean_OMIarray_NoLED = np.append(mean_OMIarray_NoLED, np.nanmean(NoLED))
    sem_OMIarray_NoLED = np.append(sem_OMIarray_NoLED, np.nanstd(NoLED)/np.sqrt(len(NoLED))) 
h, p = stats.friedmanchisquare(K_data['45'], K_data['55'], K_data['65'], K_data['75'])
print('Enhanced Friedman test Q= '+str(h))
print('Enhanced Friedman test p= '+str(p))
data = {
        "LED off": mean_OMIarray_NoLED
    }
errdata = {
        "LED off": sem_OMIarray_NoLED
    }
bar_plot(axs[2,0], data, errdata, colors=['mediumvioletred'], total_width=.8, single_width=.9, legend=False)
plt.ylabel('OMI')
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(ticks=[0, 1, 2, 3], labels=['-5', '5', '15', '25'])
plt.yticks([-0.2, 0, 0.2])
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 6 : SUPPRESSED STATS COMPARISON BAR - 4G RIGHT
plt.subplot(4,2,6)
mean_OMIarray_NoLED = []
sem_OMIarray_NoLED = []
K_data = {}
print('SUPPRESSED STATS FRIEDMAN:')
for i, dB in enumerate(np.asarray(sounds[1:])):
    NoLED = np.array((clustInfoSUPP['OMI_'+str(dB)+'_'+str(AM)+'_NoLED']))
    K_data[str(dB)] = NoLED[~np.isnan(NoLED)]
    mean_OMIarray_NoLED = np.append(mean_OMIarray_NoLED, np.nanmean(NoLED))
    sem_OMIarray_NoLED = np.append(sem_OMIarray_NoLED, np.nanstd(NoLED)/np.sqrt(len(NoLED))) 
h, p = stats.friedmanchisquare(K_data['45'], K_data['55'], K_data['65'], K_data['75'])
print('Suppressed Friedman test Q= '+str(h))
print('Suppressed Friedman test p= '+str(p))
data = {
        "LED off": mean_OMIarray_NoLED
    }
errdata = {
        "LED off": sem_OMIarray_NoLED
    }
bar_plot(axs[2,1], data, errdata, colors=['plum'], total_width=.8, single_width=.9, legend=False)
plt.ylabel('OMI')
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(ticks=[0, 1, 2, 3], labels=['-5', '5', '15', '25'])
plt.yticks([-0.2, 0, 0.2])
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 7 : ENHANCED PSTH - 4H LEFT
plt.subplot(4,2,7)
dB_sel = 55
clustInfoENH = clustInfoMask.loc[(clustInfoMask['Delta_'+str(dB_sel)+'_'+str(AM)+'_NoLED']>0)]
n = len(clustInfoENH.index)
PSTHall_NoLED_NoOdor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_NoOdor'])))
PSTHall_NoLED_Odor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_Odor'])))
meanNoLED_NoOdor = PSTHall_NoLED_NoOdor.mean(axis=0)
semNoLED_NoOdor = PSTHall_NoLED_NoOdor.std(axis=0)/np.sqrt(n)
meanNoLED_Odor = PSTHall_NoLED_Odor.mean(axis=0)
semNoLED_Odor = PSTHall_NoLED_Odor.std(axis=0)/np.sqrt(n)
plt.plot(time, smooth(meanNoLED_NoOdor, 3), color='black')
plt.plot(time, smooth(meanNoLED_Odor, 3), color='mediumvioletred')
plt.legend(['Sound (control)','Odor+Sound'])
plt.xlabel('Time (s)')
plt.ylabel('Avg. Firing Rate (Hz) Z-Scored')
plt.ylim(-5, 40)
plt.xlim(-0.25, 1.5)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.margins(x=0)
plt.xticks([0, 0.5, 1, 1.5])
plt.title(str(dB_sel-50)+ ' dB SNR')
plt.axvline(x=0, color='magenta', ls='--')
plt.axvline(x=0.250, color='gray', ls='--')
plt.axvline(x=1.250, color='gray', ls='--')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 8 : SUPPRESSED PSTH - 4H RIGHT
plt.subplot(4,2,8)
dB_sel = 75
clustInfoSUPP = clustInfoMask.loc[(clustInfoMask['Delta_'+str(dB_sel)+'_'+str(AM)+'_NoLED']<0)]
n = len(clustInfoSUPP.index)
PSTHall_NoLED_NoOdor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_NoOdor'])))
PSTHall_NoLED_Odor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_Odor'])))
meanNoLED_NoOdor = PSTHall_NoLED_NoOdor.mean(axis=0)
semNoLED_NoOdor = PSTHall_NoLED_NoOdor.std(axis=0)/np.sqrt(n)
meanNoLED_Odor = PSTHall_NoLED_Odor.mean(axis=0)
semNoLED_Odor = PSTHall_NoLED_Odor.std(axis=0)/np.sqrt(n)
plt.plot(time, smooth(meanNoLED_NoOdor, 3), color='black')
plt.plot(time, smooth(meanNoLED_Odor, 3), color='plum')
plt.legend(['Sound (control)','Odor+Sound'])
plt.xlabel('Time (s)')
plt.ylabel('Avg. Firing Rate (Hz) Z-Scored')
plt.ylim(-5, 40)
plt.xlim(-0.25, 1.5)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.margins(x=0)
plt.xticks([0, 0.5, 1, 1.5])
plt.title(str(dB_sel-50)+ ' dB SNR')
plt.axvline(x=0, color='magenta', ls='--')
plt.axvline(x=0.250, color='gray', ls='--')
plt.axvline(x=1.250, color='gray', ls='--')
plt.axhline(y=0, color='black', ls='--')

# Save Fig
filename = 'Fig 4EFGH'+'.pdf'
plt.savefig(filename, bbox_inches='tight', dpi=300, transparent=False)

plt.show()

In [ ]:
# FIGURE 4 I

fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(18, 5), sharex=False)
fig.subplots_adjust(hspace=.4,wspace=0.3)
AM = 1.0
clustInfoMask = clustInfoAll.loc[((clustInfoAll['GLM_AM_LEDoff_dB_p']<0.05) &
                             (clustInfoAll['GLM_AM_LEDoff_dB_coef']>0) &    
                            (clustInfoAll['BL_FRmean']>1) &
                                ((clustInfoAll['GLM_AM_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_AM_LEDoff_Interaction_p']<0.05)))]
clustInfoMask = clustInfoMask.reset_index(drop=True)
Resp_0_NoOdorNoLED = np.array(clustInfoMask['zResp_'+str(0)+'_'+str(AM)+'_NoLED_NoOdor'])
Resp_0_OdorNoLED = np.array(clustInfoMask['zResp_'+str(0)+'_'+str(AM)+'_NoLED_Odor'])                         
############################################################################
plt.subplot(1,3,1)
x = Resp_0_NoOdorNoLED
y = Resp_0_OdorNoLED
axmin=-10
axmax=10
plt.scatter(x, y, color='black', s=20)
plt.xlabel('control')
plt.ylabel('Odor')
plt.title('0 dB')
plt.xlim([axmin, axmax])
plt.ylim([axmin, axmax])
plt.yticks([-5, 0, 5])
plt.xticks([-5, 0, 5])
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
unity = np.linspace(axmin, axmax)
plt.plot(unity, unity, color='black', ls='--', label='_nolegend_')
t, p = stats.shapiro(x)
print('control shapiro= '+str(p))
t, pp = stats.shapiro(y)
print('Odor shapiro= '+str(pp))
s, p = stats.wilcoxon(x, y)
print(s)
print(p)
print('n= '+str(len(x)))
res = stats.wilcoxon(x, y)
print(res.statistic)
############################################################################
plt.subplot(1,3,2)
bothmeans = [x.mean(), y.mean()]
botherr = [x.std()/np.sqrt(len(x)), y.std()/np.sqrt(len(y))]
plt.bar(['control', 'Odor'], bothmeans, color=['gray', 'mediumvioletred'], edgecolor=['black', 'black'], yerr=botherr, capsize=10, fill=True)
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
plt.ylabel('Avg. Firing Rate (Hz) Z-scored')
plt.ylim([-0.1, 1.5])
plt.yticks([0, 0.5, 1, 1.5])
plt.title('0 dB')
############################################################################
plt.subplot(1,3,3)
dB_sel = 0
ind = 75
PSTHall_NoLED_NoOdor = np.squeeze(clustInfoMask.at[clustInfoMask.index[ind], 'zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_NoOdor'])
PSTHall_NoLED_Odor = np.squeeze(clustInfoMask.at[clustInfoMask.index[ind], 'zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_Odor'])
meanNoLED_NoOdor = PSTHall_NoLED_NoOdor
meanNoLED_Odor = PSTHall_NoLED_Odor
plt.plot(time, smooth(meanNoLED_NoOdor, 7), color='black')
plt.plot(time, smooth(meanNoLED_Odor, 7), color='magenta')
plt.legend(['control','Odor'], loc='upper right')
plt.xlabel('Time (s)')
plt.ylabel('Avg. Firing Rate (Hz) Z-Scored')
plt.ylim(-5, 20)
plt.xlim(-0.25, 1.5)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.margins(x=0)
plt.xticks([0, 0.5, 1, 1.5])
plt.title('0 dB')
plt.axvline(x=0, color='magenta', ls='--')
plt.axhline(y=0, color='black', ls='--')

# Save Fig
filename = 'Fig 4I'+'.pdf'
plt.savefig(filename, bbox_inches='tight', dpi=300, transparent=False)

plt.show()